In [1]:
import os
import pandas as pd
from pandas.testing import assert_frame_equal
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import numpy as np
import time
# 修改为当前的函数
from gait import *

In [ ]:
raw_data = pd.read_csv(src, skiprows=99, names=['timestamps', 'x', 'y', 'z'], usecols=[0, 1, 2, 3])
raw_data['unix_timestamps'] = pd.to_datetime(raw_data.timestamps, format="%Y-%m-%d %H:%M:%S:%f").values.astype(np.int64) // 10**6

In [2]:
def run_gaitpy(raw_data, sample_rate, subject_height, dst):
    # Load/format data
    # 下面两行是 Github 网页上没有给出的两行数据


    ### Create an instance of GaitPy ###
    gaitpy = Gaitpy(raw_data,
                    sample_rate,
                    v_acc_col_name='y',
                    ts_col_name='unix_timestamps',
                    v_acc_units='g',
                    ts_units='ms',
                    flip=False)
    
    #### Classify bouts of gait ####
    
    gait_bouts = gaitpy.classify_bouts(result_file=os.path.join(dst,'classify_bouts.h5'))
    # @阿帅批：注释掉分类
    # gait_bouts = "/Users/wangshuaibo/Documents/ScriptsofShuai/Notes/IMUStepAnalysis/gaitpy/gaitpy/demo/demo_classify_bouts.h5"
    
    #### Extract gait characteristics #### 
    # 出错在下面的位置
    gait_features = gaitpy.extract_features(subject_height,
                                            subject_height_units='centimeters',
                                            result_file=os.path.join(dst,'gait_features.csv'),
                                            classified_gait=gait_bouts)
    
    #### Plot results of gait feature extraction ####
    gaitpy.plot_contacts(gait_features, result_file=os.path.join(dst, 'plot_contacts.html'), show_plot=False)



In [3]:
def run_demo(dst):
    # Set source and destination directories
    __file__ = "Demo.py"
    #src = __file__.split(".py")[0] + "_data.csv" #官方本来的代码
    #dst = input("Please provide a path to a results directory:    ")
    while not os.path.isdir(dst):
        dst = input(
            "\nYour previous entry was not appropriate."
            "\nIt should follow a format similar to /Users/username/Desktop/Results"
            "\nPlease provide a path to a results directory:    "
        )

    # Run gaitpy
    st = time.time()
    try:
        sample_rate = 50  # hertz
        subject_height = 177  # centimeters
        # @阿帅批：运行run_gaitpy 代码
        run_gaitpy(raw_data, sample_rate, subject_height, dst)
    except Exception as e:
        print("Error processing: \nError: {}".format(e))
    stp = time.time()
    print("total run time: {} seconds".format(round(stp-st, 2)))

    # Confirm expected results
    print("Checking extract_features endpoints...")
    # @阿帅批：预期 gait_features 是官方的实例
    expected_gait_features = pd.read_csv(__file__.split(".py")[0] + '_gait_features.csv')
    # @阿帅批：obtained gait_features 是自己运行官方 demo 得到的数据
    obtained_gait_features = pd.read_csv(os.path.join(dst, 'gait_features.csv'))
    # @阿帅批：检查自己生成的 gait_features 和 demo_gait_features 是否相同
    assert_frame_equal(expected_gait_features, obtained_gait_features)
    # 检测步态分类生成数据和官方 Demo 数据是否一致
    print("Checking classify_bouts endpoints...")
    expected_classify_bouts = pd.read_hdf(__file__.split(".py")[0] + '_classify_bouts.h5')
    obtained_classify_bouts = pd.read_hdf(os.path.join(dst, 'classify_bouts.h5'))
    assert_frame_equal(expected_classify_bouts, obtained_classify_bouts)
    print("All tests passed")


In [4]:
dst = "/Users/wangshuaibo/Documents/ScriptsofShuai/Notes/IMUGaitAnalysis/ShuaiGaitpy/DIYData"

In [5]:
run_demo(dst)

	Classifying bouts of gait...
	Bout classification complete!

	Extracting features...
	Feature extraction complete!

	Plotting contacts...
	Plot complete!

total run time: 2.89 seconds
Checking extract_features endpoints...
Checking classify_bouts endpoints...


AssertionError: DataFrame.columns are different

DataFrame.columns values are different (66.66667 %)
[left]:  Index(['prediction', 'window_end_time', 'window_start_time'], dtype='object')
[right]: Index(['prediction', 'window_start_time', 'window_end_time'], dtype='object')

In [ ]:
## 单独运行步态分类 暂时不运行

In [3]:
# 单独运行步态分类
__file__ = "Demo.py"
src = __file__.split(".py")[0] + "_data.csv" 
raw_data = pd.read_csv(src, skiprows=99, names=['timestamps', 'x', 'y', 'z'], usecols=[0, 1, 2, 3])
# @阿帅批：下面这一步主要是查找对应格式的时间戳，并将其转换为 unix_timestamps 时间格式
# Unix 时间以秒为单位，方便加减进行运算
raw_data['unix_timestamps'] = pd.to_datetime(raw_data.timestamps, format="%Y-%m-%d %H:%M:%S:%f").values.astype(np.int64) // 10**6
sample_rate = 50 
gaitpy = Gaitpy(raw_data,
                sample_rate,
                v_acc_col_name='y',
                ts_col_name='unix_timestamps',
                v_acc_units='g',
                ts_units='ms',
                flip=False)

#### Classify bouts of gait ####
# @阿帅批：注释掉分类
gait_bouts = gaitpy.classify_bouts(result_file=os.path.join(dst,'classify_bouts.h5'))

	Classifying bouts of gait...
	Bout classification complete!

